# **Customer Engagement Analysis with SQL and Tableau**

Conducting a customer engagement analysis and ensuring that users experience the best of the product or service is of utmost importance for every company. In this Customer Engagement Analysis project, we use SQL to extract user behavior information from 365’s database, then feed it into Tableau to create a multi-page customer engagement dashboard. Finally, we discuss our findings and identify critical areas for improvement.

But how do we define ‘engagement’ on the 365 platform? For this project, we’ll discuss engagement in the following way. Students are engaged on a particular day if they have watched a lecture. Another term important to consider is ‘onboarding’. Students are onboarded if they have watched a lecture on the 365 platform at least once.

### **Retrieving Courses Information**

We use the tables `365_course_info`, `365_student_learning`, and `365_course_ratings` from the `365_database` to retrieve a new table with the following fields:
- `course_id` – the unique identification of a course
- `course_title` – the title of the course
- `total_minutes_watched` – all minutes watched from the course for the entire period
- `average_minutes` – all minutes watched from the course for the entire period divided by the number of students who’ve started the course
- `number_of_ratings` – the number of ratings the course has received
- `average_rating` – the sum of all the course’s ratings divided by the number of students who rated it.

We do the following steps to retrieve the desired table:

1. We start by creating a common table expression (CTE) that calculates the total minutes watched and the total number of students for each course. We use the SQL `GROUP BY` clause to group the data by each course.

2. We then create another CTE that calculates the average minutes watched for each course by using the result from the previous step (`total_minutes_watched` and `num_students`). We enclose the sub-query within parentheses and assign the alias `title_average_minutes` to this temporary result set.

3. Finally, we create a third CTE that calculates the number of ratings and the average rating for each course and group the data by course. If there are no ratings for a course, the average rating should be 0. We enclose the sub-query within parentheses and assign the alias `title_ratings` to this temporary result set. 

We retrieve the `title_ratings` result set and save it as sql-task1-courses.csv.


In [ ]:
WITH title_total_minutes AS
(
SELECT
course_id,
course_title,
ROUND(SUM(minutes_watched), 2) AS total_minutes_watched,
COUNT(DISTINCT student_id) AS num_students
FROM
365_course_info
LEFT JOIN
365_student_learning USING (course_id)
GROUP BY course_id
),

title_average_minutes AS
(
SELECT
course_id,
course_title,
total_minutes_watched,
ROUND(total_minutes_watched/num_students, 2) AS average_minutes
FROM
title_total_minutes
),

title_ratings AS
(
SELECT
course_id,
course_title,
total_minutes_watched,
average_minutes,
COUNT(course_rating) AS number_of_ratings,
ROUND(SUM(course_rating)/COUNT(course_rating), 2) AS average_rating
FROM
title_average_minutes
LEFT JOIN
365_course_ratings USING (course_id)
GROUP BY course_id
)

SELECT
course_id,
course_title,
total_minutes_watched,
average_minutes,
number_of_ratings,
average_rating
FROM
title_ratings;

### **Retrieving Purchases Information**

We now use the `365_student_purchases` table to create a new SQL query that, when executed, stores in the `365_database` schema a view called `purchases_info`, which we’ll use in subsequent tasks. This view contains information about the time intervals in which each student was a paid subscriber to our platform with the following columns:
- `purchase_id`
- `student_id`
- `purchase_type`
- `date_start` (the date the subscription started)
- `date_end` (the date the subscription ended)

To calculate the end date of a subscription (`date_end`), we add one month, three months, or 12 months to the start date of a subscription for a Monthly (represented as `0` in the `plan_id` column), Quarterly (`1`), or an Annual (`2`) purchase, respectively.

In [ ]:
DROP VIEW IF EXISTS purchases_info;

CREATE VIEW purchases_info AS
SELECT 
purchase_id,
student_id,
purchase_type,
date_purchased AS date_start,
CASE
WHEN purchase_type = 'Monthly' THEN
date_add(makedate(year(date_purchased), day(date_purchased)),
	INTERVAL MONTH(date_purchased) MONTH)
WHEN purchase_type = 'Quarterly' THEN
date_add(makedate(year(date_purchased), day(date_purchased)),
	INTERVAL MONTH(date_purchased)+2 MONTH) 
WHEN purchase_type = 'Annual' THEN
date_add(makedate(year(date_purchased), day(date_purchased)),
	INTERVAL MONTH(date_purchased)+11 MONTH) 
END AS date_end
FROM
365_student_purchases;

### **Retrieving Students Information**

Now, we use the `purchases_info` view together with the `365_student_purchases`, `365_student_info`, and `365_student_learning` tables from the database to retrieve a new temporary result set containing the following fields:

- `student_id` – a list of student IDs
- `student_country` – the country of origin they’ve entered into the platform
- `date_registered` – registration date of the students
- `date_watched` – the date they’ve watched a course
- `minutes_watched` – the minutes they’ve watched from that course on that day
- `onboarded` – whether they have a record in the `365_student_learning` table (`0` – no, `1` – yes)
- `paid` – whether they’ve had an active subscription on the day of watching the course (`0` – no, `1` – yes)

The table would include the daily minutes each student watches for each course. If they have never watched a video, they need only one entry in the table with `NULL` under `date_watched` and `0` under `minutes_watched`.

We do the following steps to retrieve the desired table:

1. We create a query that appropriately joins the `365_student_info` and the `365_student_learning` tables, retrieving all records from the first table. We select all columns from `365_student_info` and the `date_watched` column from the `365_student_learning` table. The `date_watched` column can be `NULL` if a student from the `365_student_info` hasn’t watched any lectures.

2. We create a column called `minutes_watched` to calculate the total minutes a student watches daily. If a student hasn’t watched any lectures—i.e., they don’t have any record in the `365_student_learning` table—their sum of watched minutes should be `0`.

3. We create another column called `onboarded`, evaluated at `0` if a student has no record in the `365_student_learning` table. Otherwise, it is `1`.

4. We use the query we derived as a subquery (aliased `a`), which we need to appropriately join with the `purchases_info` view so that all records from `a` are retrieved. We select all columns from `a`. Then, we create a new column called `paid`, which is evaluated at `1` if a lecture is watched between the subscription’s start date and the same subscription’s end date. Otherwise, we consider it at `0`.

5. We use the query created at step 4 as a new subquery; and call it `b`. We select the first six columns from `b`. Then, we select the maximum value of the `paid` column for a single `student_id` – `date_watched` pair. This would determine if a student was a paid member on the date specified by the `date_watched` column.

We save the retrieved result set as sql-task3-courses.csv

In [ ]:
SELECT 
    student_id,
    student_country,
    date_registered,
    date_watched,
    minutes_watched,
    onboarded,
    MAX(paid) AS paid
FROM
(SELECT 
    student_id,
    student_country,
    date_registered,
    date_watched,
    minutes_watched,
    onboarded,
    IF(date_watched BETWEEN p.date_start AND p.date_end, 1, 0) AS paid
FROM
	(SELECT 
    i.student_id,
    i.student_country,
    i.date_registered,
    l.date_watched,
	IF(l.date_watched IS NULL, 0, ROUND(SUM(l.minutes_watched), 2)) AS minutes_watched,
    IF(l.date_watched IS NULL, 0, 1) AS onboarded    
	FROM
    365_student_info i
        LEFT JOIN
    365_student_learning l USING (student_id)
	GROUP BY student_id, date_watched) a
LEFT JOIN 
    purchases_info p USING (student_id)) b
GROUP BY student_id, date_watched;

### **Interpreting the Data**

Now that we've retrieved the necessary data for our analysis, I have created a Tableau dashboard to analyze the data further.

You can find the Tableau Dashboard here:

https://public.tableau.com/views/5_Learning-Workbook2_17000054274650/Overview?:language=en-US&:display_count=n&:origin=viz_share_link

The purpose of creating this dashboard was to gain insight into customer engagement and to answer strategic questions to assess the 365 customer engagement state and find potential areas for improvement. We analyze the various key performance indicators(KPIs) as follows:

##### **1. Platform Engagement**

1.What’s the number of engaged students on the platform?

From January 1, 2022 to October 31, 2022, which is our given time period, we observe that there are a total of 19,332 engaged students.

2.How does this number differ between free plan and paying students?

The majority of the engaged students (18,032/19,332) are on a free plan, while 2,178 students had either converted to, or were originally on a paid plan during this time period.

3.How does this number change over time? Is it affected by marketing campaigns and other events?

A point to be noted is that the database we’re working with here only covers students registered between January 1 and October 31. This means that we only consider their engagement in our analysis. In general though, students registered before January 1 would also have been contributing, increasing the numbers we see at the beginning of 2022. 

Disregarding that for the time-being though, on the second page showing students daily engagement, we observe that there is a spike in the number of engaged students in the middle of August. We may infer that this occurred due to 365DataScience opening up its platform for 3 days free of charge during that time. As a result, the number of engaged users more than tripled in volume. This demonstrates the effect of free campaigns on the platform, as it results in a high student demand and considerable audience growth. 

By restricting the engagement graph to just paying users though, we observe that these 3 days didn’t result in as much of a considerable change, rather it was only the free users that experienced a drastic spike. This is likely because the users who have already payed for the product wouldn’t have had any additional incentive to engage more during the free days.

Upon returning to the all student type option and excluding the anomaly data points to study the graph without such abnormal behavior, we can see that the general trend is that the number of engaged students on the platform increases. The engagement has grown from 100 students daily in January to approximately 400 by October. Such a result is significant to report. 

Various factors contribute to this improvement of the platform, such as the new courses, exams and projects accompanying them, as well as the new features released throughout the year. Following the three free of charge days in August, the gamification of the platform in September and the marketing campaigns further helped spread the word and led to students engaging more and for longer.

This is demonstrated by the more than 30% increase in the number of engaged students per day, compared to the summer months. Upon studying the engagement for each month separately and observing the scale on the y-axis, we see the numbers increase from around 200 engaged students per day in June in July, to approximately 300 students per day in August and September to then jumping to about 400 active students in October. 

Lastly, looking at the bar chart displaying the number of engaged students by month, we can see that the last three months from the analysis are stronger in terms of the number of engaged students. It's also interesting to note the increase of active paying students as months pass.







##### **2. Onboarding and Retention**

1.What is the number of onboarding students compared to the number of registered students? 

The onboarding students cover the fraction of 365 customers who have engaged with the any of the videos, exams or quizzes. Hence, it is one of the essential metrics when assessing the level of interest students demonstrate on the platform. Onboarding is a term that each company defines differently. It's therefore difficult to set a universal benchmark for whether the onboarding rate is good or bad. 

The 365 platform hypothesizes that at least half of the registered students would engage with either of those three elements. The graph on the engagement page reveals that the onboarding rate varies between 40% and 60%. It's also apparent that the changes on the platform around mid September, have increased the onboarding rate to 70%. 

Where could this increase be coming from? On September 16, the new gamified version of the platform officially launched. We observe this positive change in the onboarding rate immediately after the launch, and hence, can conclude that the gamification caused the increase. This result confirms the initial hypothesis concerning gamification where adding perks such as coins, experience points, gifts and beautifully illustrated collections of cards encouraged 365 newcomers to engage with the content more.

2.How long do students typically stay engaged on the platform? How does this number differ between free plan students and paying ones?

We can answer this question by studying the cohort table in our dashboard. This type of analysis is most beneficial when considering free plan and paying students separately. We first select the free option on the type parameter. As expected, most of the free plan students remain engaged only within the month they've onboarded, which is also very likely the month they've registered. If we select the January cohort, we can see that only about 3% of the students that onboarded in January, continue to remain active in February. We should note of course that a fraction of these students switch to paying ones. 

3.What changes can we incorporate from a business perspective to increase the free plan students time on the platform? 

A solution we could implement is to perform a monthly retargeting of the audience who has registered in the past 30 days using paid advertisements. This could increase the engagement of the free plan students and persuade them to try out and stay on the paid version of the platform. 

Next, by selecting the option from the type parameter that filters out only the paying students, we see the general trend is such that about half of the paying students are retained after the first month and roughly 40% remain active in period two. Then we observe a plateau where approximately 30% of the students from period zero remain engaged. We can hypothesize that most students who drop out after the first month have purchased a monthly plan at the beginning of the period and cancel their subscription before the 30 days expire. The other half who remain engaged in period one are likely students who've purchased a quarterly or an annual subscription or purchased a monthly subscription at the end of the previous month. Students with annual or recurring subscriptions will most likely remain engaged in period one and later. 

We observe that the cells are darker under period zero next to January, March, July and September. These are months with a higher number of newly engaged paying subscribers, which is not surprising since each of these months is related to a marketing campaign; Winter Sale, Special Spring Offer; Most Wanted Campaign and The Data Science Hero Campaign, popularizing the new gamified look on the platform. 

What's important with the cohort analysis is that it can let the marketing team know whether they've targeted the right people during these campaigns with the knowledge, background and interests that the 365 program requires. Knowing how and where to find the right audience for your product is essential. If your customers don't engage with the product, they will likely leave soon. So it's a lose-lose situation. Both the client is left unsatisfied, and the marketing team has spent effort and money attracting the wrong crowd. If on the other hand, you find the right people for your service, namely those looking to develop their data science skill set and have the time and resources to do so, then it's a win for both sides. The customer finds the right product for them and the marketing team attracts loyal customers likely to remain subscribed for longer. That way, the customers bring more value to the company by subscribing to the premium service and very importantly, can become ambassadors for your products by recommending it to friends, colleagues or social media. 

Now, according to our data, we see that the retention curves corresponding to the January, March and July cohorts look quite similar, with 50% of the Paid students staying active in period one and only a few dropping in period two. The September cohort, however, stands out with quite a low retention rate of 35% in period one. One reason for this might be that newcomers decided to benefit from the lower price during the campaign and try the new gamified features for only a month. 

A second hypothesis is that seasonality affects people's motivation and disposition to learn new skills. During the summer, people especially college students, have more time to practice their hobbies, acquire new knowledge and expand their skills. This is in contrast to September and October, when college life resumes. Students are tasked with more responsibilities, and 365’s working audience faces a new fiscal quarter, which more often than not results in more pressure and deadlines at work. Factors like these might hinder the possibility of studying and maintaining learning streaks. 

Third, in preparing the websites new look and expanding the platform's library, the content creators of the 365 team didn't have time to create new videos for their YouTube channel. As a result, this could have reduced the audience coming from YouTube, which is often those with a genuine interest in the field who would benefit from the platform for a more extended period. Lastly, we should remember that the database we use in the project only includes users registered during 2022, while the 365 platform has had students engaging ever since 2018. 

At 365, we performed the same cohort analysis using data from 2021 and before. We find that many of the students who registered a while back had returned to the 365 platform as paying users and went on to explore the new gamified version of the platform. Additionally, many of these students have remained active during period one, thereby increasing the retention rate of the September cohort. We should then interpret our results as that 35% of the students who registered in 2022 and were first engaged as paying students in September, had also engaged with the platform in October.

##### **Content Consumption**

1.On average, how much content is consumed by students regarding minutes watched? How does the number differ between free plan students and paying ones? 

We hypothesize for this problem that the average number of minutes watched by free plan users is between 20 and 30 because this is the amount of content unlocked for all registered students from each course. Returning to the Overview page and selecting the Free option from the type parameter, we conclude that we are on the right track. This is an excellent result because it indicates the free plan students are interested in the content and in general, complete the unlocked portion of a course. Of course, some free plan students won't watch any content while others would view the free content of two or more courses. On average however, about 20 minutes of content are allocated to each  free plan student. 

How about the paying students? It turns out that this group isn't as easy to predict because each student has different learning habits and is interested in courses of different lengths. If we consider the data for all 10 months, then on average, a paying student watches about 540 minutes, amounting to approximately nine hours of content. This result makes us assume that on average, a paying student completes one extensive course. Paying students who don't follow such courses most likely enroll in a few shorter ones. The average course length on the platform is about three and a half hours. Based on the average amount they've watched and the average course length, we can conclude that a paying student completes roughly two or three courses throughout their paid membership. 

This answers are two initial questions, but our analysis can reveal even more. On the Learning page, we constructed a combo chart to analyze the trends of content consumption by month. Let's first examine the bar chart showing the number of minutes watched each month. We see that the height of the bars is gradually increasing, with August being the strongest month recording more than 240,000 minutes of content, followed by October with a little under 230,000. These results can be interpreted as follows. First, all students who registered at the beginning of the year contribute to the minutes watched in later months, leading to a gradual increase. Second, the platform hosted a lot of new registrations in August due to the three unlocked days. Third, the gamification of the platform in mid September and the following marketing campaigns have increased the number of registered and engaged students as we also saw on the engagement page. 

Let's now analyze the same plot for free plan and paying students separately. Choosing the former in the type parameter we see that the August peak is even more pronounced, with the number of minutes watched being more than three times bigger than during the other nine months. We can also see increased interest among free plan students in October. The average minutes per student per month fluctuates around 15, and has doubled in August. The same analysis for paying students looks much more different. The number of minutes monthly jumps from roughly 30,000 to approximately 150,000. Notice that the unlocked days in August haven't affected the behavior of the paying students, the same behavior we saw in the engagement charts. 

Let's now look at the line chart representing monthly minutes watched per student. The numbers there vary between 150 and 230 minutes. If consider the courses on the platform to have an average length of three and a half hours, then on average, a paying student completes one course per month. That's a great insight into our customer behavior analysis.

##### **F2P Conversion Rate and Subscription Duration**

1.What is the Free to Paid conversion rate based on the minutes watched on the platform? 

To understand this analysis better, we must remember that only a fraction of each course on the 365 platform is available to Free plan students. The rest of the content is unlocked for paying subscribers only. During their trial period, students can watch some or all free content available. So a reasonable question might be, what percentage of the free content available to students is consumed? And subsequently, we're most interested in people who've upgraded to a premium plan. We can then reformulate our question and ask how much free content have paying users watched before subscribing? 

We try to answer this question with the help of the user bucket analysis on the Learning page. We can hypothesize that the content watched up to the first hour is critical for the students interest in the program. It's therefore essential to see as precisely as possible, how the behavior changes in these first 60 minutes. Following this line of thought, the students who have watched a lot of content, such as 500 minutes and more, are expected to be very few, of significant similarity, and very unlikely to purchase a subscription. This is why the intervals in the student bucket analysis start at five minutes, then increase to 10,120, 240, 520, and finally 1000. 

By studying the chart we can see that a significant portion of the registered students don't engage in any courses. It's reasonable to assume that a customer needs to look through and experience a product to assess whether it's worth their money. Nevertheless, 3% of 365 registered customers have converted to paying ones which amounts to approximately 560 people, which isn’t an insignificant number. There could be several reasons to buy a subscription without taking advantage of the free content first. They might have benefited from an exclusive discount, watched and liked the video on the YouTube channel, been recommended the platform by a friend or colleague, or have had a free plan registration with another account and are now registering using a different one such as a company account. 

For now, we click on the first bar and exclude it from the view to see the variation in the other bars better. The next bar is the bucket of students engaged with a course for only five minutes or less. Now that we've excluded the first bucket, we can appreciate that this second bar scores another significant fraction of the registered students. As little as five minutes doesn't seem like long enough content to convince a customer to buy a subscription, as only 2.3% have done so. Still, this amounts to approximately 190 people. Such could be users who bear the same features as the previous group of people and others who have played a video only to check the quality of the stream and to make sure they like the teaching style. We do the same and remove this bar from the plot. We now observe the following behavior. The first five intervals have the same length of five minutes each. We can see that the conversion rate line chart increases, but we should consider that each bar sample size is decreasing. 

Next, we consider the bars with an interval size of 10 minutes, therefore bring more people into each bucket. We've defined nine buckets of such length. It's interesting to note that the group of people that have watched between 40 and 50 minutes is larger than the one that preceded. As a result, the free to paid conversion rate of the 40 to 50 student bucket is slightly lower. Still, the number of people converting from both buckets is roughly the same. The rest of the bar chart follows a decreasing trend. On the contrary the line chart continues to increase. We should make note of the peek at the 110 to 120 bucket and the bucket sitting to the left and right of this one which have very high conversion rates. We see that almost half of the students who've watched about two hours of content, have also purchased a subscription. That's a significant observation. Out of nearly 900 people entering these three buckets, more than 370 convert to paying students, a success rate of about 40%. An interesting analysis that could be performed is to find the courses which students entering this bucket watch before converting, and then complete as paying students. These could be offered to newly registered free plan students since these courses have the highest conversion probability. 

Lastly, let's look at the four final bars of the chart. For these last couple of buckets, we've increased the interval of watch minutes to 240, 520, and 1000. This time, both the bar and the line charts drop. We compare the [240 - 480] bucket with the [90 - 100] bucket, since they have almost the same number of representatives. The conversion rate of the former is about 16%, while the latter is approximately 28%, a significant difference. 

So what conclusions can we draw from this analysis? With a few exceptions, students are more likely to pay for full access when they find a course that meets their expectations. It's therefore of utmost importance to introduce students right from the start to a course they might enjoy. One idea is to design a quiz that upon completion, recommend the user a course to get them started based on their interest and advancement in the field. Another possibility is to grant students the first 24 hours on the platform for free in the form of an A/B test and a B test implies that this 24 hour unlocked period would be available to only a fraction of the students. The hypothesis is that this change would help this group of people explore the content better, get involved in a course and reach those two hours of watched content that presumably converts students best. 

Finally, we look at the second student bucket visualization that analyzes the average subscription duration based on the minutes watched on the platform. From studying the line chart, it’s good to see that on average, the more content students consume, the longer they stay on the platform. Very few students exert strange behavior such as watching a lot of content for a short period or watching almost no content while staying subscribed for long.

##### **Courses Engagement**
- 
    1.Which are the most watched and enjoyed courses on the platform? 
    
    To answer this, we record three metrics. The first one is the overall minutes watched from a course. From the dashboard, we can see that the five courses sitting at the top of the list are Introduction to Data and Data Science, SQL statistics, Introduction to Excel, and Python Programmer Bootcamp. Unsurprisingly, these courses are the ones that students engage with the most and frequently ask questions in the Q&A hub. The Introduction to Data and Data Science course is the obvious choice for someone who's just embarked on a data science journey. It builds the fundamentals for all other topics covered on the platform. Even though it's on the shorter side with less than two hours of content,  its still being at the top of the list is suggestive of its importance and high demand. In fact, all five courses in the top five list cover the fundamentals required for a data science or data analysis career. Two courses on the list, SQL and Python Programmer Bootcamp, are the longest on the platform with more than 8 and almost 11 hours of content respectively. This gives them an advantage in summing the watched minutes. However, this is true to only some extent. The creators of these courses extensively cover two of the most important programming languages in data science in a professional yet engaging way. The fact that Python and SQL are two of the most watched courses on the platform despite their comprehensiveness, tells us that the students appreciate the way of teaching. The other courses on the list, Statistics and Introduction to Excel, are also critical introductory courses covering topics essential for an aspiring data scientist. Interestingly, among the other theoretical courses on the platform, such as Mathematics and Probability, Statistics seems to be the most in demand. 
    
    The second metric is minutes watched per student. In this top five list, we see similar titles; SQL and Python programmer bootcamp, with about 110 minutes allocated to each student who enrolls. Other courses that students find interesting and have roughly the same metric value are those connected to Data Pre-processing and Data Literacy, which are fundamental topics in data science. We also draw our attention to the negotiation course. Unlike the other four courses, which are longer than the average course length on the platform (3.5 hrs), the negotiation course is about an hour and a half long, making it an outlier in this list. It was however, added to the 365 library in the middle of September, which is towards the end of the period considered in this project’s database. This means that not many people would have had the chance to watch it by the end of October. Additionally, negotiation is a member of the fifth module, which contains courses only available to students at level eight or above. This makes for very few people who would’ve gotten access to it by the end of October. A quick check in the database reveals that one student had gotten the chance to unlock and also complete the course. We therefore expect negotiation to be at the top of the completion rate list. 
    
    We now choose the completion rate option. This metric was defined as the minutes watched per student divided by the course length, and it was presented in percentages. As we predicted, the negotiation course leads the score with a 95% completion rate. This is followed by the marketing strategy course, which only five students had unlocked and enrolled. Following at a 50% completion rate are two relatively short courses on the platform. The data literacy course is another one entering the top five list as an outlier regarding course length. Therefore, we were on the right track with our initial hypothesis. This list would feature relatively new and short courses.
    
    Finally, let's look at the results from the donut chart. The average rating of the courses on the platform is evaluated at 4.8, with almost 2300 5-star ratings out of more than 2700 votes. This is a great result and shows that the 365 platform upgrading efforts have payed off. From a strategic point of view, this analysis can show which courses are performing well and pinpoint a potential problem with a particular course. So reporting low ratings for a course or a low engagement rate, can point toward an issue with product quality.

##### **Exams**

Let us now examine the Exams and Certificates page of the dashboard. The questions we’ll be looking at are:

1.What's the number of exams taken on the platform? 

2.What is student's general exam success rate on the different types of exams, including practice, course, and career track?

 Let's start with the left hand side of the page, with the horizontal stacked bar chart showing the split between successful and unsuccessful exam attempts monthly. On average, the number of monthly exams started is 2500, with August being an outlier having more than 7000 exams attempted, and October as the next in line with more than 5000. Of course, some months report a higher exam attempt rate than others, but the general trend for the entire period is increased numbers. Next, we can study students exam performance. A 60% score or more grant students a pass. The practice exams don't weigh in on the students course grade though, they serve only as a preparation for the course exam at the end of the course. This means that a student wouldn't receive an official grade from a practice exam, but would still receive a score. Upon selecting the practice exam category, the bar chart displays definitive results. Roughly 30% of the monthly practice exams don't meet the 60% passing benchmark. The number is slightly higher during August and October when more practice exams are taken.

Let's now examine the performance in course exams. The number of course exams taken compared to the practice ones is smaller, which makes sense since on average, each course has about two or three practice exams, and strictly one course exam. Like the practice exams, roughly 70% of the course exams have a passing bar, with August recording about 60% due to the higher number of exams attempted. This can be explained by considering that some students sit for a course exam intending to obtain a certificate without having first completed the lectures from the course. This could often lead to the student failing the exam. Such cases are more frequent during free days, like the ones in August. Note also that the x-axis doesn't correspond to the number of people who have attempted an exam, but rather the number of exams that have been tried. Therefore, these results include several attempts of the same exam by the same person, which might be misleading. 

Finally, let's study the performance on the career track exams. The results are much more different compared to the two previous ones. First, the number of career track exams attempted each month varies between 5 and 15, where August is again an outlier with 36 attempts. This tells us that most of the students on the platform enter to pass one or several individual courses without intending to follow a specific career track. It might also be the case that students are discouraged or don't dedicate as much time to completing an entire track. The next point is that the fraction of exams passed is not as high indicating that the career track exams require an excellent understanding of the material of all seven courses included. This result also implies that the time to solve the exam might be shorter. Before increasing the allowed time, however, an additional analysis should first be conducted that studies how much time students spend to solving a career track exam.

##### **Certificates**
- In this section, we’ll be looking at the following:
    
    1. The number of course and career track certificates issued
    
    2. The fraction of the students who enroll in a career track and complete it
    
    We can study the number of certificates issued on the overview page, where we created a corresponding key performance indicator. Looking at the entire period we have data for, we see about 3600 certificates issued, including course and career track. We can break this result down into months by going to exams and certificates page and concentrating on the right hand side of the page. The number of course certificates has risen as the year progressed. One of the factors is that students who registered early in the year also contribute to the later months. From the cohort analysis, however, we've seen that the engagement of paying students, the ones who can issue a certificate drops down to 50% after the first month, then to about 40% after the second and third months, and then plateaus at about 25 to 30%. Therefore, that factor alone cannot explain the rise in certificate issuance. Another one could be that the number of registered students increases monthly and new students gradually contribute. Additionally, as we've seen on the engagement page, student engagement has a positive trend. 
    
    We notice that the tendency of this bar chart is the same as that of the one representing course exam attempts. To prove that let us exclude the unsuccessful attempts from the analysis. Both charts show an increase from January to March, then a slight decrease until May. Then we notice an increase in the summer months with June and July having bars of almost equal heights followed by the outlier of the analysis, and then returning to normal with October being the more active month. The fact that these two charts are of such considerable similarity is not surprising. Passing a course exam automatically granted the students a course exam certificate. However, the number of exams passed doesn't perfectly match the number of certificates issued. For example, 99 passed course exams in January, but only 91 certificates were issued. There might be several reasons for such discrepancies. 
    
    First, the issuance of a certificate might be delayed sometimes and issued the following month relative to the month of the exam completion. Second, there are cases when the certificate holders name needs to be corrected. Upon the students request, the certificate is invalidated and a new one is issued which might happen in the following month. Thirdly, of course, exams could be retaken. Occasionally, students pass an exam but want to improve their grades and retake it, which changes the grading system but doesn't issue another certificate. 
    
    Let's now look at how the numbers look when we filter out only the career track certificates. We see that not only is the height of the bars much smaller, but not all months are present on the chart. No career track certificates are issued in January and February, as there haven't been any track exams attempted during these months from students who signed up that same year. We've seen that it takes our students around a month on average to complete an entire course. A career track requires passing nine courses before sitting for the exam. This means we'd expect the average career track journey to take roughly nine months. Of course students who dedicate more time to the program compared to the average can try for a career certificate sooner than that. All right. 
    
    Let’s now address the second point, what fraction of students who enroll in a career track complete it. Let us first click the career track funnel button to reveal a new chart. Without filtering the data, we see that out of 7900 people who enroll, less than 12% attempt a course exam included in the track. A significant number of the students who haven't entered this second part might come from the ones who registered during the free days in August. We see however, that once a student attempts a course exam they are very likely to complete it, since the difference between the second and third bar is only 1%. Next comes the career track exam. To be allowed to sit for such an exam, a student should have completed all nine course exams, the ones from the seven compulsory and two elective courses. This requires a lot of work and dedication which is why only 0.94% of the enrolled students, or less than 9% of those who've completed at least one course exam, reached this fourth step of attempting a track exam. Finally, only 43 out of all 7900 enrolled, passed the track exam, which is about 60% of those who attempted it. This corresponds well with the bar chart on the left hand side of the page, where approximately 50% of the attempted track exams have been passed. A 10% discrepancy is unsurprising since a track exam could be attempted several times. Therefore the number of those who've attempted an exam is smaller than the number of attempts themselves. We can analyze this chart for each track individually. The Careers the students are most interested in include data analyst, data scientist, and business analysts respectively. The completion rate however, is in precisely the opposite order. More than 1% of the students who enroll in the business analyst track complete it, which is 12 students in total. This is followed by the data scientists with 0.53% or 16 students in total. And finally, the certified data analysts amounted to only 0.38% of the enrolled students, totaling 15. 
    
    This analysis concludes that most enrolled students give up already on the first step requiring a student to attempt an exam from the track. The 365 team released the gamified version of the platform to motivate their students to study and keep an eye on their streaks. They’ve also added a social element to help students share their difficulties and seek help from fellow students. Inevitably, the path toward a career track is long and paved with difficulties. In response, 365 sought to be there to help answer questions and equip with a positive mindset required to obtain a career track certificate.


##### **Further Steps and Improvement**

-    
    Let us now discuss some ideas on further attributes our dashboard could feature. For the front page, we could think of other KPIs from which the company could benefit. For example, the marketing team could ask for the content consumed during the entire period alongside the number of engaged students, the number of minutes per student and the number of certificates issued on the platform. Such statistics are great for marketing campaign advertisements, because those numbers speak well to the audience. Next, we go to the course metrics horizontal bar charts. Such a chart would greatly benefit from a date filter, which could help track the progress of the courses. The same could be implemented for the donut chart. If it doesn't become too overcrowded, we could also include a parameter that allows for choosing a specific course. 
    
    Next, we go to the engagement and onboarding analysis page. Here we implemented several options; choosing the start and end dates, the month and the year. We could try adding a fourth option that allows for choosing to filter by week, or even better, incorporate all these views in one so the dashboard user can access them quickly. Additionally, notice the wave like form of the engagement line for paying users. We could try switching the metric on the X-axis from date to days, and studying the days of the week students are most active. This could for example, tell us about the most appropriate day for new feature releases when student engagement is expected to be higher. Finally, for this page, something else we could play around with is the summaries and trend lines in the analytics pane in Tableau. There we can add constant and average lines, and display the median and quartiles. Under the model option, we can include different trend lines depending on our problem. 
    
    The third page we constructed concerns student retention, visualized with the help of a cohort analysis table and retention curves for each cohort. Most often when we're dealing with retention analysis, we need the statistics in percentages. Our table however, has the disadvantage of showing the absolute number of students in each period, rather than their fraction. We can select the cohort we're interested in and see the percentages displayed on the corresponding line chart. Still reading can be more challenging if three or more line charts overlap. This is why this page could benefit from a second cohort table view where the numbers inside the cells store the percentage values we have on the line chart. Additionally, we could add the possibility to granulate the data further and display students retention weekly rather than monthly. Another functionality we could introduce in our dashboard is reducing the number of periods displayed on the table, and choosing which cohort to display at the top. These filters are currently not crucial since the small cohort table fits on the screen. But as the years pass by, the number of periods and cohorts will start growing until there is no space for the numbers inside the cells. Next, as we mentioned in our exams analysis, exams can be retaken. We could add a parameter that filters out exams attempted for the first time, second time and more. This would allow for studying the difference in success rates. The possibility of retaking an exam also means that this visualization doesn't tell us the number of people who've attempted an exam, but rather only the number of exams attempted. We could therefore add a chart that displays this information or a chart that displays the number of exams per student each month, just as we've done for the number of minutes watched. Finally, we could add the option for a daily or weekly view. 
    
    One of the visualizations on the right hand side of this page displays the number of monthly certificates. Like the other plots, we could play around with the parameters and add the option to choose different date granularities. Another feature we could add is filtering the data by course or course library module. We could also create a stacked bar chart with the top part displaying the course with most certificates issued, while the bottom represents the rest of the certificates. We could think of anything so long as it's functional, intuitive and most importantly, valuable and informative. The second visualization on the right hand side is the career track funnel. Earlier, we mentioned adding a parameter that sifts out students enrolled at a particular time. In addition, we could put in more funnel layers and make the jump from the first level to the second one less prominent. One way in which the funnel can be modified is to add the following; Enrolled in a track, started a lecture from a course in the career track, attempted a practice exam from a course in the track, watched an entire course from the track, attempted a course exam, completed a course exam, completed more than one but not all course exams from the track, completed all course exams from the track, attempted a track exam, passed a track exam and hence earned a certificate. This could help us understand the student's journey and learn which stage hinders most students from reaching the career track exam. 
    
    Finally, we go to the Student Learning page. On the left hand side, we can again add the option to change the date granularity. On the right hand side, we have the bucket analysis. During our discussion, we mentioned different changes to the charts such as trying out different interval sizes or creating a histogram at equal intervals, realized in Tableau with the help of the custom bins option. Last, with the hover tooltips, we could add one for each visualization and describe how each chart works, what they represent, and whether there is something that dashboard users should be aware of.